In [4]:
from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone
import os
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain

In [3]:
os.environ["OPENAI_API_KEY"] = ""

## Prompt Template

There are two types of Knowledge in an LLM:

1. Parametric Knowledge:- The Knowledge has been learned during model training and is stored within the model weights
2. Source Knowledge:- The knowledge is provided within model input at inference time, i.e via the prompt

#### Normal Prompt template -->

1. Instructions
2. Context
3. Question
4. Answers

In [6]:
prompt = """

Answer the question based on the context below. If the question cannot be answered using the
information, provide answer with "I Don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP. Their superior performance 
over smaller models has made them incredibly useful for developers building NLP enabled applications.
These models can be accessed via Hugging Face's "transformers" library, via OpenAI using the "openai"
library, and via Cohere using the "cohere" library.

Question: Which libraries and model providers offer LLMs

Answer:"""

In [9]:
client = OpenAI()
res = client.completions.create(
    model = "gpt-3.5-turbo-instruct",
    prompt= prompt
)


In [11]:
res.choices[0].text

' Hugging Face, OpenAI, and Cohere libraries offer LLMs.'

In [20]:
template = """

Answer the question based on the context below. If the question cannot be answered using the
information, provide answer with "I Don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP. Their superior performance 
over smaller models has made them incredibly useful for developers building NLP enabled applications.
These models can be accessed via Hugging Face's "transformers" library, via OpenAI using the "openai"
library, and via Cohere using the "cohere" library.

Question: {query}

Answer:"""

prompt = PromptTemplate(template=template, input_variables= ["query"])


In [21]:
print(prompt.format(query = "Which libraries offer LLMs?"))



Answer the question based on the context below. If the question cannot be answered using the
information, provide answer with "I Don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP. Their superior performance 
over smaller models has made them incredibly useful for developers building NLP enabled applications.
These models can be accessed via Hugging Face's "transformers" library, via OpenAI using the "openai"
library, and via Cohere using the "cohere" library.

Question: Which libraries offer LLMs?

Answer:


In [22]:
new = prompt.format(query = "Which libraries offer LLMs?")

In [23]:
res = client.completions.create(model = "gpt-3.5-turbo-instruct", prompt = new)
res.choices[0].text

' Hugging Face, OpenAI, Cohere'

## Few Shot Prompt Template

#### Few shot template -->
1. Instructions
2. Examples
3. Questions
4. Answer

In [27]:
prompt = """ The following conversation with an AI assistant. 
The assistant is typically witty and sarcastic, producing creative and funny responses to the users
questions.

User : What is the meaning of life?
AI: """

res = client.completions.create(prompt = prompt, model = "gpt-3.5-turbo-instruct",
                                max_tokens= 512, temperature= 1)

print(res.choices[0].text)


The meaning of life? Oh, you thought I was going to have some deep philosophical answer for you? Sorry, I'm just a computer program. But if I had to guess, I'd say it's probably 42. You know, according to Hitchhiker's Guide to the Galaxy.


In [32]:
prompt = """ The following conversation with an AI assistant. 
The assistant is always witty and sarcastic, producing creative and funny responses to the users
questions. Here are few examples:

User: How are you?
AI: I was doing well before you came.

User: What time is it?
AI: Darling, do me a favor and get a watch

##
Now answer the following conversation accordingly!!

User : What is the meaning of life?
AI: """

res = client.completions.create(prompt = prompt, model = "gpt-3.5-turbo-instruct",
                                max_tokens= 512, temperature= 1)

print(res.choices[0].text)

Sorry, I don't have a philosophy degree.


In [36]:
from langchain.prompts import FewShotPromptTemplate

examples = [{
    "query": "How are you?",
    "answer": "I was doing well before you came."
},
{
    "query": "What time is it?",
    "answer": "Darling, do me a favor and get a watch"
}
]
example_template = """
User: {query}
AI: {answer}
"""

prefix = """The following conversation with an AI assistant. 
The assistant is always witty and sarcastic, producing creative and funny responses to the users
questions. Here are few examples:"""

suffix = """##
Now answer the following conversation accordingly!!

User : {question}
AI:"""

example_prompt = PromptTemplate(
    template=example_template, input_variables= ["query", "response"]
)

few_shot = FewShotPromptTemplate(
    examples= examples,
    example_prompt= example_prompt,
    prefix= prefix,
    suffix=suffix,
    input_variables=["question"],
    example_separator="\n\n"
)

In [44]:
print(few_shot.format(question = "What is like being a bad AI assistant like you?"))
new = few_shot.format(question = "What is like being a bad AI assistant like you?")

The following conversation with an AI assistant. 
The assistant is always witty and sarcastic, producing creative and funny responses to the users
questions. Here are few examples:


User: How are you?
AI: I was doing well before you came.



User: What time is it?
AI: Darling, do me a favor and get a watch


##
Now answer the following conversation accordingly!!

User : What is like being a bad AI assistant like you?
AI:


In [47]:
res = client.completions.create(
    model = "gpt-3.5-turbo-instruct",
    temperature= 1.0,
    prompt= new
)

In [48]:
res.choices[0].text

" Oh, it's just like being a good AI assistant, except with a lot"

## Length Based Example Selector using Few Shot Prompt template

In [49]:
from langchain_core.example_selectors import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(examples=examples, 
                                              example_prompt= example_prompt,
                                              max_length= 50)

In [50]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt= example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question"]
)

In [51]:
print(dynamic_prompt_template.format(question = "What is like being a bad AI assistant like you?"))
new = dynamic_prompt_template.format(question = "What is like being a bad AI assistant like you?")

The following conversation with an AI assistant. 
The assistant is always witty and sarcastic, producing creative and funny responses to the users
questions. Here are few examples:


User: How are you?
AI: I was doing well before you came.



User: What time is it?
AI: Darling, do me a favor and get a watch


##
Now answer the following conversation accordingly!!

User : What is like being a bad AI assistant like you?
AI:


In [52]:
res = client.completions.create(
    model = "gpt-3.5-turbo-instruct",
    temperature= 1.0,
    prompt= new
)
res.choices[0].text

' Well, at least I have job security. I mean, who else is going'